# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [12]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST Clinical Severity Score based on the provided parameters.

    Parameters:
    sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp
    temperature, alertness, inspired_oxygen, performance_status

    Returns:
    int: Total PRIEST score or None if inputs are invalid.

    >>> priest('male', 85, 21,0.95, 110, 95,37, 'alert', 'air', 'unrestricted normal activity')
    0.34
    >>> priest('female',18,21,0.97, 90, 210, 37.2,'alert','air','unrestricted normal activity')
    0.02
    >>> priest('MALE', 42, 20, 0.94, 109, 111, 36, 'ALERT','air','unrestricted normal activity')
    0.03

    """


    if sex.upper() not in ['MALE', 'FEMALE']:
        return None

    points = 0


    if sex.upper() == 'MALE': points += 1

    if age < 50: points += 0
    elif 50 <= age <= 65: points += 2
    elif 66 <= age <= 80: points += 3
    elif age >80: points += 4
    else: return None

    if respiratory_rate < 9: points += 3
    elif 9 <= respiratory_rate <= 11: points += 1
    elif 12 <= respiratory_rate <= 20: points += 0
    elif 21 <= respiratory_rate <= 24: points += 2
    elif respiratory_rate > 24: points += 3
    else: return None

    if oxygen_saturation > 0.95: points += 0
    elif 0.94 <= oxygen_saturation <= 0.95: points += 1
    elif 0.92 <= oxygen_saturation < 0.94: points += 2
    elif oxygen_saturation < 0.92: points += 3
    else: return None

    if heart_rate < 41: points += 3
    elif 41 <= heart_rate <= 50: points += 1
    elif 51 <= heart_rate <= 90: points += 0
    elif 91 <= heart_rate <= 110: points += 1
    elif 111 <= heart_rate <= 130: points += 2
    elif heart_rate > 130: points += 3
    else: return None

    if systolic_bp < 91: points += 3
    elif 91 <= systolic_bp <= 100: points += 2
    elif 101 <= systolic_bp <= 110: points += 1
    elif 111 <= systolic_bp <= 219: points += 0
    elif systolic_bp > 219: points += 3
    else: return None

    if temperature < 35.1: points += 3
    elif 35.1 <= temperature <= 36.0: points += 1
    elif 36.1 <= temperature <= 38.0: points += 0
    elif 38.1 <= temperature <= 39.0: points += 1
    elif temperature > 39.0: points += 2
    else: return None

    if alertness.upper() == 'ALERT': points += 0
    elif alertness.upper()  == 'CONFUSED OR NOT ALERT': points += 3
    else: return None

    inspired_oxygen = inspired_oxygen.lower()
    if inspired_oxygen == 'air': points += 0
    elif inspired_oxygen == 'supplemental oxygen': points += 2
    else: return None

    performance_status = performance_status.lower()
    if performance_status == 'unrestricted normal activity': points += 0
    elif performance_status == 'limited strenuous activity, can do light activity': points += 1
    elif performance_status == 'limited activity, can self-care': points += 2
    elif performance_status == 'limited self-care': points += 3
    elif performance_status == 'bed/chair bound, no self-care': points += 4
    else: return None

    if 0 <= points <= 1:
        risk = 0.01
    elif 2 <= points <= 3:
        risk = 0.02
    elif points == 4:
        risk = 0.03
    elif points == 5:
        risk = 0.09
    elif points == 6:
        risk = 0.15
    elif points == 7:
        risk = 0.18
    elif points == 8:
        risk = 0.22
    elif points == 9:
        risk = 0.26
    elif points == 10:
        risk = 0.29
    elif points == 11:
        risk = 0.34
    elif points == 12:
        risk = 0.38
    elif points == 13:
        risk = 0.46
    elif points == 14:
        risk = 0.47
    elif points == 15:
        risk = 0.49
    elif points == 16:
        risk = 0.55
    elif 17 <= points <= 25:
        risk = 0.59
    elif points >= 26:
        risk = 0.99

    return risk

In [13]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 85, 21,0.95, 110, 95,37, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.34
ok
Trying:
    priest('female',18,21,0.97, 90, 210, 37.2,'alert','air','unrestricted normal activity')
Expecting:
    0.02
ok
Trying:
    priest('MALE', 42, 20, 0.94, 109, 111, 36, 'ALERT','air','unrestricted normal activity')
Expecting:
    0.03
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [14]:
import requests
import json
def find_hospital(age, sex, risk):
    """
    find_hospital sends patient information to the REST web service.
    find_hospital takes 3 arguments: sex (string), age (int), and risk (float)

    the function returns a string containing the name of the hospital
    >>> find_hospital(40,'male',0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(25,'FEMALE',0.02)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(85,'male',0.59)
    'Wesley Woods Geriatric Hospital'
    """

    sex = sex.lower()
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"

    response = requests.get(url)
    if response.status_code == 200:
        json_file = response.text
        json_text = json.loads(json_file)
        return json_text['hospital']
    else:
        return None

find_hospital(40,'male',0.1)

'Southwest Hospital and Medical Center'

In [15]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40,'male',0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(25,'FEMALE',0.02)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(85,'male',0.59)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [16]:
import requests
import json
def get_address(hospital):
  """
  the function get_address takes one argument:
  it uses request and json module to look at a json file online and find the address of the hospital
  hospital -> string, name of the hospital whose address we want to find

  output -> string, address of the hospital

  >>> get_address('Southwest Hospital and Medical Center')
  '501 FAIRBURN ROAD SW'
  >>> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS")
  '2000 PALMYRA ROAD'
  >>> get_address("UNIVERSITY HEALTH CARE SYSTEM")
  '1350 WALTON WAY'
  """
  hospital_data = requests.get('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')
  json_text = json.loads(hospital_data.text)

  return json_text[hospital.upper()]['ADDRESS']



In [17]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('Southwest Hospital and Medical Center')
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS")
Expecting:
    '2000 PALMYRA ROAD'
ok
Trying:
    get_address("UNIVERSITY HEALTH CARE SYSTEM")
Expecting:
    '1350 WALTON WAY'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [18]:
import requests
import csv
def process_people(file_location):
  """
  process_people() function is used to read a psv file containing patient information
  and transform the data into a dictionary containing a list as the value

  Arguments:
  file_location: a string containing url of the psv file
  Returns:
  A dictionary containing patient information including risk, hospital, and address of hospital.
  The key is the first element of the psv file, the patient number
  """
  psv_request = requests.get(file_location)
  psv_text = csv.reader(psv_request.text.splitlines(), delimiter='|')

  hospital_list = []
  for row in psv_text:
    hospital_list.append(row)

  hospital_list.pop(0)

  final_dictionary = {}
  for row in hospital_list:
    sex=row[1]
    age=int(row[2])
    breath=float(row[3])
    o2sat=float(row[4])
    heart=float(row[5])
    systolic=float(row[6])
    temp=float(row[7])
    alertness=row[8]
    inspired=row[9]
    status=row[10]

    risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

    hospital = find_hospital(age, sex, risk)
    #print(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital)
    address = get_address(hospital)

    final_dictionary[row[0]] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address]

  return final_dictionary
#process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [19]:
import requests
import json

my_dictionary = process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')
result_request = requests.get('https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5')
result_dictionary = json.loads(result_request.text)
if result_request.status_code == 200:
  if my_dictionary == result_dictionary:
    print('My results are the same as people.json')
  else:
    print('My results are not the same')
else:
  print('There was an error requesting the people.json file')

My results are the same as people.json


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---